<a href="https://colab.research.google.com/github/sejongsmarcle/2023_Spring_Kaggle_Study/blob/main/%EB%B2%84%EC%8A%A4_%EC%9A%B4%ED%96%89%EC%8B%9C%EA%B0%84_%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#matplotlib 사용할 때 한국어 깨지지 않게 하기 위한 처리
!apt -qq -y install fonts-nanum > /dev/null

import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'

font = fm.FontProperties(fname=fontpath, size=9)

fm._rebuild()

%config InlineBackend.figure_format = 'retina'

plt.rc('font', family='NanumBarunGothic')  
#필요한 한국어 폰트를 다운받아서 원하는 경로에 위치시키는 것

In [2]:
#필요한 라이브러리 불러오기기
import pandas as pd
import lightgbm as lgb
import folium

#데이터 불러오기
train = pd.read_csv('/train.csv')
test = pd.read_csv('/test.csv')
submission = pd.read_csv('/submission_제출양식.csv')

In [3]:
train.head() #route_id: 버스 별로 고유의 번호(reute_nm을 다른 숫자로 치환)

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude,next_arrive_time
0,0,2019-10-15,405136001,7997025,360-1,33.456267,126.551750,제주대학교입구,06시,266.0,제대마을,33.457724,126.554014,24
1,1,2019-10-15,405136001,7997025,360-1,33.457724,126.554014,제대마을,06시,333.0,제대아파트,33.458783,126.557353,36
2,2,2019-10-15,405136001,7997025,360-1,33.458783,126.557353,제대아파트,06시,415.0,제주대학교,33.459893,126.561624,40
3,3,2019-10-15,405136001,7997025,360-1,33.479705,126.543811,남국원(아라방면),06시,578.0,제주여자중고등학교(아라방면),33.484860,126.542928,42
4,4,2019-10-15,405136001,7997025,360-1,33.485662,126.494923,도호동,07시,374.0,은남동,33.485822,126.490897,64


In [4]:
submission.head()

,id,next_arrive_time
0,210457,0
1,210458,0
2,210459,0
3,210460,0
4,210461,0


In [6]:
train.tail()

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude,next_arrive_time
210452,210452,2019-10-28,405328102,7983486,281-2,33.255783,126.577450,비석거리,21시,528.0,삼아아파트,33.251896,126.574417,96
210453,210453,2019-10-28,405328102,7983486,281-2,33.248595,126.568527,동문로터리,21시,280.0,매일올레시장 7번입구,33.249753,126.565959,50
210454,210454,2019-10-28,405328102,7983486,281-2,33.251891,126.560303,서귀포시 구 버스터미널,21시,114.0,아랑조을거리 입구,33.251084,126.559551,16
210455,210455,2019-10-28,405328102,7983486,281-2,33.251084,126.559551,아랑조을거리 입구,21시,223.0,평생학습관,33.249504,126.558068,38
210456,210456,2019-10-28,405328102,7983486,281-2,33.248487,126.511195,대륜동주민센터,21시,189.0,서귀포버스터미널(가상정류소),33.249091,126.509224,24


In [8]:
print(train['date'].min())
print(train['date'].max())
print(test['date'].min())
print(test['date'].max())

2019-10-15
2019-10-28
2019-10-29
2019-11-05


In [10]:
print(train.shape)  #파일의 행과 열 개수 출력  #지금까지 학습한 데이터에 비해 크기가 엄청남
print(test.shape)
print(submission.shape)

(210457, 14)
(91774, 13)
(91774, 2)


In [11]:
train.info() #모든 컬럼이 완전하게 채워져 있음(이번 데이터에는 결측값 없음)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210457 entries, 0 to 210456
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                210457 non-null  int64  
 1   date              210457 non-null  object 
 2   route_id          210457 non-null  int64  
 3   vh_id             210457 non-null  int64  
 4   route_nm          210457 non-null  object 
 5   now_latitude      210457 non-null  float64
 6   now_longitude     210457 non-null  float64
 7   now_station       210457 non-null  object 
 8   now_arrive_time   210457 non-null  object 
 9   distance          210457 non-null  float64
 10  next_station      210457 non-null  object 
 11  next_latitude     210457 non-null  float64
 12  next_longitude    210457 non-null  float64
 13  next_arrive_time  210457 non-null  int64  
dtypes: float64(5), int64(4), object(5)
memory usage: 22.5+ MB


In [12]:
test.info() #test도 모든 컬럼들이 온전하게 채워져 있음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91774 entries, 0 to 91773
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               91774 non-null  int64  
 1   date             91774 non-null  object 
 2   route_id         91774 non-null  int64  
 3   vh_id            91774 non-null  int64  
 4   route_nm         91774 non-null  object 
 5   now_latitude     91774 non-null  float64
 6   now_longitude    91774 non-null  float64
 7   now_station      91774 non-null  object 
 8   now_arrive_time  91774 non-null  object 
 9   distance         91774 non-null  float64
 10  next_station     91774 non-null  object 
 11  next_latitude    91774 non-null  float64
 12  next_longitude   91774 non-null  float64
dtypes: float64(5), int64(3), object(5)
memory usage: 9.1+ MB


In [13]:
train.describe() #평균적으로 다음 정류장까지 가는데 걸리는 시간이 85초(1분 25초)

,id,route_id,vh_id,now_latitude,now_longitude,distance,next_latitude,next_longitude,next_arrive_time
count,210457.000000,2.104570e+05,2.104570e+05,210457.000000,210457.000000,210457.000000,210457.000000,210457.000000,210457.000000
mean,105228.000000,4.052491e+08,7.988694e+06,33.434528,126.603451,490.256100,33.434711,126.603687,85.380824
std,60753.847139,9.132404e+04,6.774077e+03,0.102350,0.123961,520.563932,0.102224,0.123838,85.051170
min,0.000000,4.051360e+08,7.983000e+06,33.244382,126.473300,97.000000,33.244382,126.473300,6.000000
25%,52614.000000,4.051365e+08,7.983093e+06,33.325283,126.523900,291.000000,33.325283,126.524550,44.000000
50%,105228.000000,4.053201e+08,7.983431e+06,33.484667,126.551050,384.000000,33.484860,126.551050,66.000000
75%,157842.000000,4.053201e+08,7.997041e+06,33.500197,126.650322,542.000000,33.500228,126.650322,102.000000
max,210456.000000,4.053281e+08,7.997124e+06,33.556167,126.935188,7461.000000,33.556167,126.935188,2996.000000


In [14]:
test.describe()

,id,route_id,vh_id,now_latitude,now_longitude,distance,next_latitude,next_longitude
count,91774.000000,9.177400e+04,9.177400e+04,91774.000000,91774.000000,91774.000000,91774.000000,91774.000000
mean,256411.051038,4.052462e+08,7.988904e+06,33.437067,126.600925,479.771689,33.437304,126.601197
std,26531.098040,9.184224e+04,6.818353e+03,0.101350,0.123887,491.476531,0.101221,0.123745
min,210457.000000,4.051360e+08,7.983005e+06,33.244382,126.473300,97.000000,33.244382,126.473300
25%,233441.250000,4.051365e+08,7.983093e+06,33.340329,126.521348,291.000000,33.344884,126.523759
50%,256415.500000,4.053201e+08,7.983432e+06,33.485199,126.547770,381.000000,33.485199,126.547770
75%,279388.750000,4.053201e+08,7.997056e+06,33.500228,126.642733,529.000000,33.500473,126.642733
max,302354.000000,4.053281e+08,7.997124e+06,33.556167,126.935188,7461.000000,33.556167,126.935188


groupby() 
===
집단별로 통계랑 확인 가능

상당히 많은 그룬들이 있어서 많이 쓰일 예정

In [15]:
len(train['now_station'].unique())

348

In [17]:
train[['route_id', 'vh_id']].drop_duplicates() #중복 되는 것들 없애기기

,route_id,vh_id
0,405136001,7997025
181,405136001,7997034
381,405136001,7997035
539,405136001,7997040
717,405136001,7997044
...,...,...
20226,405136522,7997004
26037,405320116,7983225
27098,405320122,7983067
45506,405328102,7983401


In [18]:
train[['route_id', 'vh_id']].drop_duplicates().groupby('route_id').count()  #각각의 노션별로 운행되고 있는 차량의 개수 확인 가능

,vh_id
route_id,
405136001,10
405136002,5
405136007,1
405136012,3
405136521,6
405136522,15
405320111,3
405320112,5
405320113,1


In [19]:
train[['route_id', 'next_arrive_time']].groupby('route_id').mean()  #노선 별로 다음 정류장까지 가는 데에 걸리는 시간 #매우 상이하다

,next_arrive_time
route_id,
405136001,92.063688
405136002,108.270852
405136007,94.172947
405136012,111.322633
405136521,114.673939
405136522,101.339188
405320111,68.901710
405320112,67.602392
405320113,68.403649


folium.Map()
====
지도 시각화 패키지
도화지 위에다가 그림을 겹겹이 쌓아 올리는 것

In [20]:
coord = [35.1559861, 129.1384361]  #부산 시의 좌표
map_osm = folium.Map(location = coord) #위치를 함수를 이용하여 변수에 저장 #전세계 지도 지원함
map_osm

In [22]:
map_osm = folium.Map(location = coord, zoom_start=15) #기본이 10 #지도 확대시킬 수 있음
map_osm

In [23]:
map_osm = folium.Map(location = coord, tiles = 'Stamen Terrain') #map에 마우스 두고 ctrl+스페이스바 누르면 지도 유형 선택 가능 #terrain: 산지, 산모형
map_osm

In [24]:
map_osm = folium.Map(location = coord, tiles = 'Stamen Toner') #흑백 지도
map_osm

In [27]:
coord = [35.2335123, 129.0810047]

map_osm = folium.Map(location = coord, zoom_start = 12)

folium.Marker(coord, popup='부대부대', tooltip = '여긴 어디게').add_to(map_osm) #마커 위에 마커를 클릭했을 시 팝업 시킬 안내 문구는 popup #마우스 후버링 할 때는 팝업하는 문구는 tooltip #마커 생성 뒤 지도에 추가

map_osm.save('index.html') #코랩에서 할 때, 한국어로 잘 보고 싶을 때

map_osm

In [32]:
map_data = train[['route_id', 'now_latitude', 'now_longitude', 'now_station']]
map_bus_route = map_data[map_data['route_id'] == 405136521].drop_duplicates("now_station")

map_osm = folium.Map(location=[33.4134, 126.5190], zoom_start = 10.5)

for item in map_bus_route.index:    #모든 정류장을 반복문을 통해 지도에 찍어봄
  lat = map_bus_route.loc[item, 'now_latitude']
  long = map_bus_route.loc[item, 'now_longitude']

  folium.Marker([lat, long], popup = map_bus_route.loc[item, 'now_station'],
                icon = folium.Icon(color = 'red', icon = 'info-sign')).add_to(map_osm)

map_osm.save('index.html')

map_osm

전처리
===
이번에는 결측값이 없으므로 기존의 값들을 모델이 학습 가능한 형태로 바꿀 것(문자 수치화)

In [37]:
#now_arrive_time 문자열을 수치화 #슬라이싱 하기 #마지막을 자르기(인덱스는 -1) #dtype 확인, 숫자도 수치형으로 바꾸기
train['now_arrive_time'] = train['now_arrive_time'].str.slice(stop = -1).astype('int')

In [38]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210457 entries, 0 to 210456
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                210457 non-null  int64  
 1   date              210457 non-null  object 
 2   route_id          210457 non-null  int64  
 3   vh_id             210457 non-null  int64  
 4   route_nm          210457 non-null  object 
 5   now_latitude      210457 non-null  float64
 6   now_longitude     210457 non-null  float64
 7   now_station       210457 non-null  object 
 8   now_arrive_time   210457 non-null  int64  
 9   distance          210457 non-null  float64
 10  next_station      210457 non-null  object 
 11  next_latitude     210457 non-null  float64
 12  next_longitude    210457 non-null  float64
 13  next_arrive_time  210457 non-null  int64  
dtypes: float64(5), int64(5), object(4)
memory usage: 22.5+ MB


In [39]:
train.head()

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude,next_arrive_time
0,0,2019-10-15,405136001,7997025,360-1,33.456267,126.551750,제주대학교입구,6,266.0,제대마을,33.457724,126.554014,24
1,1,2019-10-15,405136001,7997025,360-1,33.457724,126.554014,제대마을,6,333.0,제대아파트,33.458783,126.557353,36
2,2,2019-10-15,405136001,7997025,360-1,33.458783,126.557353,제대아파트,6,415.0,제주대학교,33.459893,126.561624,40
3,3,2019-10-15,405136001,7997025,360-1,33.479705,126.543811,남국원(아라방면),6,578.0,제주여자중고등학교(아라방면),33.484860,126.542928,42
4,4,2019-10-15,405136001,7997025,360-1,33.485662,126.494923,도호동,7,374.0,은남동,33.485822,126.490897,64


In [40]:
test['now_arrive_time'] = test['now_arrive_time'].str.slice(stop = -1).astype('int')

In [41]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91774 entries, 0 to 91773
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               91774 non-null  int64  
 1   date             91774 non-null  object 
 2   route_id         91774 non-null  int64  
 3   vh_id            91774 non-null  int64  
 4   route_nm         91774 non-null  object 
 5   now_latitude     91774 non-null  float64
 6   now_longitude    91774 non-null  float64
 7   now_station      91774 non-null  object 
 8   now_arrive_time  91774 non-null  int64  
 9   distance         91774 non-null  float64
 10  next_station     91774 non-null  object 
 11  next_latitude    91774 non-null  float64
 12  next_longitude   91774 non-null  float64
dtypes: float64(5), int64(4), object(4)
memory usage: 9.1+ MB


변수 선택 및 모델 구축
===


In [44]:
features = ['now_latitude', 'now_longitude', 'now_arrive_time', 'distance']
target = ['next_arrive_time']
x_train, x_test, y_train = train[features], test[features], train[target]  #xtrain과 ytrain을 토대로 모델 학습시키고 xtest로 성능 확인하기

In [45]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

In [48]:
model_dict = {
    'linear':LinearRegression(),
    'rf':RandomForestRegressor(random_state=0, n_jobs=-1),
    'lgbm':lgb.LGBMRegressor(random_state=0)
}

In [49]:
model_dict

{'linear': LinearRegression(),
 'rf': RandomForestRegressor(n_jobs=-1, random_state=0),
 'lgbm': LGBMRegressor(random_state=0)}

In [50]:
model_dict.keys()  #모델에 있는 키들 출력

dict_keys(['linear', 'rf', 'lgbm'])

In [52]:
model_dict['linear']

LinearRegression()

In [54]:
model_result = {}
for key in model_dict.keys():
  print("훈련 중")
  model_dict[key].fit(x_train, y_train)
  print("예측 중")
  model_result[key] = model_dict[key].predict(x_test)

훈련 중
예측 중
훈련 중


<ipython-input-54-89433e67b85d>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_dict[key].fit(x_train, y_train)


예측 중
훈련 중
예측 중


In [56]:
model_result.keys()

dict_keys(['linear', 'rf', 'lgbm'])

In [58]:
model_result['lgbm']
model_result['linear']
model_result['rf']

array([ 35.36650507,  45.12237491, 135.88134374, ..., 126.37972036,
        31.84467172,  88.46184286])

In [60]:
#제출 위해 답안지에 옮겨 적기
lr_sub = submission.copy()
rf_sub = submission.copy()
lgbm_sub = submission.copy()

In [62]:
lr_sub['next_arrive_time'] = model_result['linear']
rf_sub['next_arrive_time'] = model_result['rf']
lgbm_sub['next_arrive_time'] = model_result['lgbm']

In [64]:
 lgbm_sub.head()  #값 답안지에 잘 들어감

,id,next_arrive_time
0,210457,42.124006
1,210458,60.248783
2,210459,120.812412
3,210460,61.146493
4,210461,150.454103


In [66]:
lr_sub.to_csv('lr_submit.csv', index = False)
rf_sub.to_csv('rf_submit.csv', index = False)
lgbm_sub.to_csv('lgbm_submit.csv', index = False)